In [1]:
from spatialdata import SpatialData
import anndata as ad
from insitupy import read_xenium
from pathlib import Path

In [2]:
from spatialdata import SpatialData
import pandas as pd
from geopandas import GeoDataFrame
import anndata as ad

def convert_insitu_to_spatialdata(insitu_data):
    """
    Convert an InSituData object to a SpatialData object.
    
    Parameters:
        insitu_data (InSituData): The InSituData object to convert.
        
    Returns:
        SpatialData: The converted SpatialData object.
    """
    
    # 1. Images
    images = {
        "nuclei": insitu_data.images.nuclei  # Ensure this is in a compatible format, like a 2D numpy array.
    }
    
    # 2. Labels
    labels = {}  # Add segmentation labels here if available, otherwise leave it empty.
    
    # 3. Points
    if isinstance(insitu_data.transcripts, pd.DataFrame):
        points = {"transcripts": insitu_data.transcripts}
    else:
        raise ValueError("Expected 'transcripts' to be a pandas DataFrame with spatial coordinates.")
    
    # 4. Shapes
    # Access nuclear and cellular boundaries as attributes instead of dictionary-style indexing.
    shapes = {
        "nuclear": GeoDataFrame(insitu_data.cells.boundaries.nuclear),
        "cellular": GeoDataFrame(insitu_data.cells.boundaries.cellular)
    }
    
    # 5. Tables
    if isinstance(insitu_data.cells.matrix, ad.AnnData):
        tables = {"cell_data": insitu_data.cells.matrix}
    else:
        raise ValueError("Expected 'cells.matrix' to be an AnnData object.")
    
    # Construct SpatialData object with the prepared elements
    spatial_data = SpatialData(
        images=images,
        labels=labels,
        points=points,
        shapes=shapes,
        table=tables
    )
    
    # Optionally, add metadata from InSituData
    spatial_data.metadata = {
        'method': insitu_data.method,
        'slide_id': insitu_data.slide_id,
        'sample_id': insitu_data.sample_id,
        'path': insitu_data.path,
        'metadata_file': insitu_data.metadata_file
    }
    
    return spatial_data




In [3]:
out_dir = Path("demo_dataset") # output directory
data_dir = out_dir / "output-XETG00000__0001879__Replicate 1" # directory of xenium data
image_dir = out_dir / "unregistered_images" # directory of images
xd = read_xenium(data_dir)

In [4]:
xd.load_all()

Loading annotations...
No `annotations` modality found.
Loading cells...
Loading images...
Loading regions...
No `regions` modality found.
Loading transcripts...


In [5]:
convert_insitu_to_spatialdata(xd)

C:\Users\Aitana\AppData\Local\Temp\ipykernel_141424\3829264614.py:45: DeprecationWarning: `table` is being deprecated as an argument to `SpatialData.__init__.__init__` in spatialdata version 0.1.0, switch to `tables` instead.
  spatial_data = SpatialData(


TypeError: Unsupported type <class 'list'>

In [8]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\Aitana\OneDrive\Documentos\Github\InSituPy\notebooks\demo_dataset\output-XETG00000__0001879__Replicate 1
Metadata file:	experiment.xenium
    ➤ images
       nuclei:	(25778, 35416)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               nuclear
               cellular
    ➤ transcripts
       DataFrame with shape 42638083 x 7